In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/splitdata/test.h5
/kaggle/input/splitdata/__results__.html
/kaggle/input/splitdata/train.h5
/kaggle/input/splitdata/__notebook__.ipynb
/kaggle/input/splitdata/__output__.json
/kaggle/input/splitdata/custom.css
/kaggle/input/combine-dataset/__results__.html
/kaggle/input/combine-dataset/combine_data.h5
/kaggle/input/combine-dataset/__notebook__.ipynb
/kaggle/input/combine-dataset/__output__.json
/kaggle/input/combine-dataset/custom.css
/kaggle/input/combine-data/combine_data.h5


In [2]:
def set_display(
    row_col_flag=True,width_flag=False,max_rows=None,
    max_columns=None,max_colwidth=None,width=None,reset_flag=False):
    if reset_flag:
        pd.reset_option('display.max_rows')
        pd.reset_option('display.max_columns')
        pd.reset_option('display.max_colwidth')
        pd.reset_option('display.width')
    else:
        if row_col_flag:
            # Hiển thị toàn bộ dòng
            pd.set_option('display.max_rows', max_rows)
            # Hiển thị toàn bộ cột
            pd.set_option('display.max_columns', max_columns)
        if width_flag:
            # Hiển thị đầy đủ nội dung trong mỗi ô (tránh bị "...")
            pd.set_option('display.max_colwidth', max_colwidth)
            # Tắt giới hạn độ rộng khung hiển thị (rộng hết khung notebook)
            pd.set_option('display.width', width)
set_display()

In [3]:
data=pd.read_hdf('/kaggle/input/combine-dataset/combine_data.h5',key='df')
test=pd.read_hdf('/kaggle/input/splitdata/test.h5',key='df')
data.head()

,user_id,course_id,enroll_time,course_name,course_prerequisites,course_about,course_field,school_id,school_name,school_about,school_motto,teacher_id,teacher_name,teacher_about,teacher_job_title,teacher_org_name,video_id,video_ccid,video_duration,user_gender,watch_id,total_watch_time,actual_watch_time,exercise_id,exercise_num_problem,doing_exercise_id,doing_num_correct,concept_id,concept_prerequisite,concept_name,concept_field,course_total_comments,user_course_num_comment,user_course_num_replies
0,U_24,C_597314,1560360127,university chinese—analysis of northern song d...,,"at present, the talent gap in data science and...",[chinese language and literature],"[S_1, S_14]","[tsinghua university, southeast university]",[referred tsinghua short directly ministry edu...,"[selfimprovement virtue carry thing, stop high...",[T_113],[wang bugao],[wang bugao yangzhong jiangsu studied nanjing ...,[distinguished professor],[tsinghua university],"[V_10741, V_10729, V_10669, V_10636, V_10583, ...","[059F9E3D0C61EC3A9C33DC5901307461, 05BA5BEC636...","[678.88, 1861.28, 280.63, 1488.369, 1192.15, 6...",1,NaN,NaN,NaN,"[Ex_10607, Ex_10647, Ex_10628, Ex_10711, Ex_10...","[1, 6, 4, 2, 4, 3, 2, 6, 3, 4, 5, 6, 4, 2, 2, ...",NaN,NaN,"[K_D_中国语言文学, K_爱国豪情_中国语言文学, K_爱国将领_中国语言文学, K_爱...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[d, patriotic pride, patriotic general, patrio...","[chinese language and literature, chinese lang...",0,0,0
1,U_24,C_597208,1560784961,data mining: theory and algorithm,,the most interesting theory + the most useful ...,[computer science and technology],[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,[selfimprovement virtue carry thing],[T_28],[yuan bo],[doctor computer science university queensland...,[associate researcher],[tsinghua university],"[V_3025, V_2985, V_2964, V_3074, V_3082, V_306...","[03D205A6AEB0B16F9C33DC5901307461, 03F5AF209FA...","[1003.9, 773.66, 886.35, 505.57, 889.61, 930.6...",1,NaN,NaN,NaN,"[Ex_2994, Ex_3099, Ex_3066, Ex_3021, Ex_3006, ...","[5, 3, 5, 3, 5, 6, 3, 3, 5, 5, 4, 5, 5, 2, 5, ...",NaN,NaN,"[K_0 part_计算机科学与技术, K_2 cities_计算机科学与技术, K_2d ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[part, cities, 2d space, matrix, + 2, 3d infor...","[computer science and technology, computer sci...",0,0,0
2,U_24,C_682129,1570379031,software theory basics,,"this course introduces formal language, automa...",NaN,[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,[selfimprovement virtue carry thing],[T_2648],[luo guiming],[luo guiming received master degree university...,[professor],[tsinghua university],"[V_265396, V_265411, V_265404, V_265442, V_265...","[00784179599D8C659C33DC5901307461, 036F26D5314...","[1236.79, 1212.67, 1584.55, 1574.35, 376.79, 5...",1,NaN,NaN,NaN,"[Ex_265490, Ex_265468, Ex_265398, Ex_265475, E...","[4, 5, 5, 6, 6, 6, 7, 7, 6, 6, 6, 5, 7, 7, 6, 5]",NaN,NaN,"[K_ASCII_计算机科学与技术, K_cfl的判定性质_计算机科学与技术, K_CYK算...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[ascii, determination of the nature of cfl, cy...","[computer science and technology, computer sci...",13,0,0
3,U_24,C_674910,1570557064,pattern aesthetics and creation,,[national premium course] this course provides...,NaN,[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,[selfimprovement virtue carry thing],[T_307],[nie yuehua],[nie yuehua member chinese artist association ...,[associate professor],[tsinghua university],NaN,NaN,NaN,1,NaN,NaN,NaN,"[Ex_35603, Ex_35595, Ex_35599, Ex_35583, Ex_35...","[1, 1, 2, 3, 1, 1, 3, 2, 2, 2, 3]",NaN,NaN,NaN,NaN,NaN,NaN,386,0,0
4,U_24,C_707038,1570971985,organic chemistry,,the world's organic compounds are rich and col...,"[chemical engineering and technology, chemical]",[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,[selfimprovement virtue carry thing],"[T_999, T_1000, T_1001, T_1002, T_1003, T_1004...","[li yanmei, li zhaolong, hua ruimao, fu hua, z...",[li yanmei

In [4]:
print(f"number of user: {data['user_id'].nunique()}\nnumber of course: {data['course_id'].nunique()}")

number of user: 367103
number of course: 2766


# Tách dữ liệu của khóa học từ dữ liệu tổng hợp

In [5]:
course_cols = data.columns[data.columns.str.contains('^(?:course|school|teacher|video|exercise|concept)', regex=True)].tolist()
course_df=data[course_cols].groupby('course_id').first()
course_df['count_course'] = data.groupby('course_id').size()
course_df=course_df.reset_index()
display(course_df.shape,course_df.head())

(2766, 25)

,course_id,course_name,course_prerequisites,course_about,course_field,school_id,school_name,school_about,school_motto,teacher_id,teacher_name,teacher_about,teacher_job_title,teacher_org_name,video_id,video_ccid,video_duration,exercise_id,exercise_num_problem,concept_id,concept_prerequisite,concept_name,concept_field,course_total_comments,count_course
0,C_1017355,introduction to traditional chinese medicine e...,the main subjects are college students who hav...,traditional chinese medicine culture - the tre...,None,[S_33],[guangzhou university traditional chinese medi...,[guangzhou university chinese medicine first b...,[nan],"[T_7970, T_7971, T_5826, T_7978, T_7979]","[su hong, liu shuaishuai, zhang xiaohu, liu, m...",[master english linguistics currently dean sch...,"[associate professor, lecturer, associate prof...",[guangzhou university traditional chinese medi...,None,None,None,"[Ex_1617861, Ex_1736572, Ex_1617873, Ex_161788...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...",None,None,None,None,39624,1940
1,C_1017419,smart government talk,,this course combines specific cases of traffic...,None,[S_52],[central south university nationality],[southcentral minzu university referred scmzu ...,[nan],"[T_7849, T_16199, T_16200, T_16201, T_16202, T...","[hu xinli, wu kaisong, peng qingjun, fang fuji...",[doctor management associate professor directo...,"[associate professor, professor, professor, as...","[central south university nationality, central...",None,None,None,"[Ex_1618263, Ex_1618257, Ex_1618248, Ex_161826...","[15, 15, 20, 15, 15, 15, 15, 16, 15]",None,None,None,None,609,128
2,C_1025064,principles of ideological and political education,,"hello, dear classmate! do you want to know how...",None,[S_13],[central south university],[central south university located changsha cit...,[integration knowledge action practice world],"[T_3915, T_3911, T_5426, T_5427, T_5429]","[liu wei, liu yongwei, lin jie, guo pengfei, c...",[liu wei male born hengnan county hunan provin...,"[associate professor, lecturer, lecturer, lect...","[central south university, central south unive...","[V_1622637, V_1622640, V_1622622, V_1622571, V...","[0B63D5E3D4D034E89C33DC5901307461, 0B6DEBC0F80...","[466.575, 402.325, 476.65, 1025.9, 755.075, 42...","[Ex_1622651, Ex_1622617, Ex_1622586, Ex_162263...","[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1...",None,None,None,None,696,1366
3,C_1025076,advanced roller skating tutorial,,the true meaning of experiencing roller skatin...,None,[S_72],[northeastern university],[northeastern university northeastern universi...,[nan],[T_5394],[li zhongshan],[fulltime teacher roller skating program north...,[lecturer],[northeastern university],None,None,None,"[Ex_1622870, Ex_1622876, Ex_1622875, Ex_1622889]","[1, 1, 15, 1]",None,None,None,None,2,165
4,C_1025079,scientific fitness method-silk-winding fitness...,none,silk wrapping skills are the chinese solution ...,None,[S_97],[beijing sport university],[beijing sport university referred beijing spo...,[nan],[T_7939],[wu dong],[professor phd education phd supervisor famous...,[professor],[beijing sport university],"[V_1623000, V_1622990, V_1622999, V_1623003, V...","[0805D199A89E1D799C33DC5901307461, 1026CDC4B72...","[671.97, 925.17, 748.11, 644.25, 543.93, 546.3...","[Ex_1623011, Ex_1622993, Ex_1623006, Ex_162302...","[5, 5, 5, 5, 5]",None,None,None,None,125,688


In [6]:
import pandas as pd

def is_effectively_null(x):
    if x is None or (isinstance(x, float) and pd.isna(x)):
        return True
    if isinstance(x, list):
        return any(pd.isna(i) for i in x)
    return False

def compute_null_stats(df):
    cols = df.columns
    isnull_rate = df.isnull().mean()

    effectively_null_rate = {}
    for col in cols:
        # Nếu kiểu dữ liệu của cột là list
        non_null_values = df[col].dropna()
        if not non_null_values.empty and isinstance(non_null_values.iloc[0], list):
            effectively_null_rate[col] = df[col].dropna().apply(is_effectively_null).mean()
        else:
            effectively_null_rate[col] = 0.0

    null_df = pd.DataFrame({
        'column': cols,
        'isnull_rate': [round(isnull_rate[col], 6) for col in cols],
        'effectively_null_rate': [round(effectively_null_rate[col], 6) for col in cols]
    })
    return null_df

null_stats = compute_null_stats(course_df)
display(null_stats)

,column,isnull_rate,effectively_null_rate
0,course_id,0.000000,0.000000
1,course_name,0.000000,0.000000
2,course_prerequisites,0.000000,0.000000
3,course_about,0.000000,0.000000
4,course_field,0.837672,0.000000
5,school_id,0.014461,0.000000
6,school_name,0.014461,0.000000
7,school_about,0.014461,0.000000
8,school_motto,0.014461,0.587307
9,teacher_id,0.005785,0.000000


Trong dữ liệu về khóa học:

Các cột kiểu danh sách chứa phần tử là giá trị khuyết: school_motto, teacher_name, teacher_about, teacher_job_title, video_ccid, video_duration, concep_preprequisite. (do các cột này bị khuyết từ metadata)
 - Thực hiện bổ sung bằng giá trị phổ biến nhất trong cột (đối với cột kiểu phân loại, văn bản), giá trị trung bình (đối với dữ liệu kiểu số)

Các cột chứa giá trị null: course_field, các cột school, teacher, video, exercise, concept (do khóa học không có liên kết với loại entity đó) => tìm top k thực thể phổ biến nhất (k là trung bình số thực thể liên kết với khóa học)
 - Đầu tiên điền các cột id đó bằng top k id phổ biến trong dữ liệu.
 - Tiếp theo bổ sung các giá trị thuộc tính theo id đã điền.


In [7]:
# Hàm xử lý danh sách, thay giá trị khuyết bằng "unknown"
def fill_missing_in_list(lst,value='unknown'):
    if isinstance(lst, list):
        return [value if pd.isna(item) else item for item in lst]
    return lst  # Giữ nguyên nếu không phải list
    
# Chuẩn hóa giá trị về string và xử lý điều kiện
course_df['course_prerequisites'] = course_df['course_prerequisites'].astype(str).apply(
    lambda x: "no prerequisite" if x.strip() == "" or x.strip().lower().startswith("no") else x
)
course_df['concept_prerequisite'] = course_df['concept_prerequisite'].apply(
    lambda x: fill_missing_in_list(x, value="no prerequisite")
)
# Danh sách các cột cần xử lý
list_columns = [
    'school_about',
    'school_motto',
    'teacher_name',
    'teacher_about',
    'teacher_job_title',
    'teacher_org_name'
]



# # Áp dụng cho từng cột trong danh sách
# for col in list_columns:
#     course_df[col] = course_df[col].apply(fill_missing_in_list)


In [8]:
display(course_df.shape,course_df.head())

(2766, 25)

,course_id,course_name,course_prerequisites,course_about,course_field,school_id,school_name,school_about,school_motto,teacher_id,teacher_name,teacher_about,teacher_job_title,teacher_org_name,video_id,video_ccid,video_duration,exercise_id,exercise_num_problem,concept_id,concept_prerequisite,concept_name,concept_field,course_total_comments,count_course
0,C_1017355,introduction to traditional chinese medicine e...,the main subjects are college students who hav...,traditional chinese medicine culture - the tre...,None,[S_33],[guangzhou university traditional chinese medi...,[guangzhou university chinese medicine first b...,[nan],"[T_7970, T_7971, T_5826, T_7978, T_7979]","[su hong, liu shuaishuai, zhang xiaohu, liu, m...",[master english linguistics currently dean sch...,"[associate professor, lecturer, associate prof...",[guangzhou university traditional chinese medi...,None,None,None,"[Ex_1617861, Ex_1736572, Ex_1617873, Ex_161788...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...",None,None,None,None,39624,1940
1,C_1017419,smart government talk,no prerequisite,this course combines specific cases of traffic...,None,[S_52],[central south university nationality],[southcentral minzu university referred scmzu ...,[nan],"[T_7849, T_16199, T_16200, T_16201, T_16202, T...","[hu xinli, wu kaisong, peng qingjun, fang fuji...",[doctor management associate professor directo...,"[associate professor, professor, professor, as...","[central south university nationality, central...",None,None,None,"[Ex_1618263, Ex_1618257, Ex_1618248, Ex_161826...","[15, 15, 20, 15, 15, 15, 15, 16, 15]",None,None,None,None,609,128
2,C_1025064,principles of ideological and political education,no prerequisite,"hello, dear classmate! do you want to know how...",None,[S_13],[central south university],[central south university located changsha cit...,[integration knowledge action practice world],"[T_3915, T_3911, T_5426, T_5427, T_5429]","[liu wei, liu yongwei, lin jie, guo pengfei, c...",[liu wei male born hengnan county hunan provin...,"[associate professor, lecturer, lecturer, lect...","[central south university, central south unive...","[V_1622637, V_1622640, V_1622622, V_1622571, V...","[0B63D5E3D4D034E89C33DC5901307461, 0B6DEBC0F80...","[466.575, 402.325, 476.65, 1025.9, 755.075, 42...","[Ex_1622651, Ex_1622617, Ex_1622586, Ex_162263...","[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1...",None,None,None,None,696,1366
3,C_1025076,advanced roller skating tutorial,no prerequisite,the true meaning of experiencing roller skatin...,None,[S_72],[northeastern university],[northeastern university northeastern universi...,[nan],[T_5394],[li zhongshan],[fulltime teacher roller skating program north...,[lecturer],[northeastern university],None,None,None,"[Ex_1622870, Ex_1622876, Ex_1622875, Ex_1622889]","[1, 1, 15, 1]",None,None,None,None,2,165
4,C_1025079,scientific fitness method-silk-winding fitness...,no prerequisite,silk wrapping skills are the chinese solution ...,None,[S_97],[beijing sport university],[beijing sport university referred beijing spo...,[nan],[T_7939],[wu dong],[professor phd education phd supervisor famous...,[professor],[beijing sport university],"[V_1623000, V_1622990, V_1622999, V_1623003, V...","[0805D199A89E1D799C33DC5901307461, 1026CDC4B72...","[671.97, 925.17, 748.11, 644.25, 543.93, 546.3...","[Ex_1623011, Ex_1622993, Ex_1623006, Ex_162302...","[5, 5, 5, 5, 5]",None,None,None,None,125,688


In [9]:
# school_motto, teacher_name, teacher_about, teacher_job_title
def fill_null(df, col, count_col='count_course'):
    """
    Điền khuyết cho cột `col` bằng giá trị có tổng `count_col` lớn nhất.
    
    Args:
        df (pd.DataFrame): DataFrame cần xử lý.
        col (str): Tên cột cần điền khuyết.
        count_col (str): Cột đếm dùng để tính tổng (mặc định là 'course_count').

    Returns:
        pd.DataFrame: DataFrame sau khi điền khuyết.
    """
    entity_df=df[[col,count_col]].dropna().explode(col)
    # Tìm giá trị xuất hiện nhiều nhất dựa trên tổng count_col
    value = entity_df.groupby(col)[count_col].sum().idxmax()
    # Điền khuyết
    df[col] = df[col].apply(lambda x: fill_missing_in_list(x, value=value))
    
cols_list = ['school_motto', 'teacher_name', 'teacher_about', 'teacher_job_title']

for col in cols_list:
    print(f'Thực hiện điền khuyết cột {col}')
    fill_null(course_df,col)

Thực hiện điền khuyết cột school_motto
Thực hiện điền khuyết cột teacher_name
Thực hiện điền khuyết cột teacher_about
Thực hiện điền khuyết cột teacher_job_title


In [10]:
print('Sau khi thay thế:')
null_stats = compute_null_stats(course_df)
display(null_stats)

Sau khi thay thế:


,column,isnull_rate,effectively_null_rate
0,course_id,0.000000,0.0
1,course_name,0.000000,0.0
2,course_prerequisites,0.000000,0.0
3,course_about,0.000000,0.0
4,course_field,0.837672,0.0
5,school_id,0.014461,0.0
6,school_name,0.014461,0.0
7,school_about,0.014461,0.0
8,school_motto,0.014461,0.0
9,teacher_id,0.005785,0.0


In [11]:
from collections import Counter

def replace_null_id(df, col='course_field', count_col='count_course'):
    def shorten_string(s, max_len=50):
        if isinstance(s, str) and len(s) > max_len:
            return s[:max_len//2 - 1] + '...' + s[-max_len//2 + 2:]
        return s

    print(f'Điền khuyết cho cột {col}')
    
    valid_rows = df[df[col].notnull()]

    # Tính số nhãn mỗi dòng
    num_labels_per_row = valid_rows[col].apply(lambda x: len(x) if isinstance(x, list) else 0)
    average_labels = num_labels_per_row.mean()
    print(f'Số {col} trung bình trên mỗi khóa học: {average_labels}')
    
    k = round(average_labels)

    # Gom tất cả nhãn, nhân theo count_course
    all_labels = []
    print(f'Tìm top-{k} {col} ...')
    for _, row in valid_rows.iterrows():
        labels = row[col]
        count = row[count_col] if count_col in row and pd.notna(row[count_col]) else 1
        if isinstance(labels, list):
            all_labels.extend(labels * int(count))

    # Đếm tần suất có trọng số
    label_counts = Counter(all_labels)
    
    # Chọn top-k nhãn phổ biến nhất
    top_k_labels = [shorten_string(label) for label, _ in label_counts.most_common(k)]

    # Thay thế các giá trị null bằng top_k_labels
    df[col] = df[col].apply(lambda x: x if isinstance(x, list) else top_k_labels)


In [12]:
col_list = ['course_field','school_id','teacher_id']
for col in col_list:
    replace_null_id(course_df,col)

Điền khuyết cho cột course_field
Số course_field trung bình trên mỗi khóa học: 1.2561247216035634
Tìm top-1 course_field ...
Điền khuyết cho cột school_id
Số school_id trung bình trên mỗi khóa học: 1.0077035950110051
Tìm top-1 school_id ...
Điền khuyết cho cột teacher_id
Số teacher_id trung bình trên mỗi khóa học: 3.8745454545454545
Tìm top-4 teacher_id ...


In [13]:
print('Sau khi thay thế:')
null_stats = compute_null_stats(course_df)
display(null_stats)

Sau khi thay thế:


,column,isnull_rate,effectively_null_rate
0,course_id,0.000000,0.0
1,course_name,0.000000,0.0
2,course_prerequisites,0.000000,0.0
3,course_about,0.000000,0.0
4,course_field,0.000000,0.0
5,school_id,0.000000,0.0
6,school_name,0.014461,0.0
7,school_about,0.014461,0.0
8,school_motto,0.014461,0.0
9,teacher_id,0.000000,0.0


In [14]:
course_df.head()

,course_id,course_name,course_prerequisites,course_about,course_field,school_id,school_name,school_about,school_motto,teacher_id,teacher_name,teacher_about,teacher_job_title,teacher_org_name,video_id,video_ccid,video_duration,exercise_id,exercise_num_problem,concept_id,concept_prerequisite,concept_name,concept_field,course_total_comments,count_course
0,C_1017355,introduction to traditional chinese medicine e...,the main subjects are college students who hav...,traditional chinese medicine culture - the tre...,[computer science and technology],[S_33],[guangzhou university traditional chinese medi...,[guangzhou university chinese medicine first b...,[selfimprovement virtue carry thing],"[T_7970, T_7971, T_5826, T_7978, T_7979]","[su hong, liu shuaishuai, zhang xiaohu, liu, m...",[master english linguistics currently dean sch...,"[associate professor, lecturer, associate prof...",[guangzhou university traditional chinese medi...,None,None,None,"[Ex_1617861, Ex_1736572, Ex_1617873, Ex_161788...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...",None,None,None,None,39624,1940
1,C_1017419,smart government talk,no prerequisite,this course combines specific cases of traffic...,[computer science and technology],[S_52],[central south university nationality],[southcentral minzu university referred scmzu ...,[selfimprovement virtue carry thing],"[T_7849, T_16199, T_16200, T_16201, T_16202, T...","[hu xinli, wu kaisong, peng qingjun, fang fuji...",[doctor management associate professor directo...,"[associate professor, professor, professor, as...","[central south university nationality, central...",None,None,None,"[Ex_1618263, Ex_1618257, Ex_1618248, Ex_161826...","[15, 15, 20, 15, 15, 15, 15, 16, 15]",None,None,None,None,609,128
2,C_1025064,principles of ideological and political education,no prerequisite,"hello, dear classmate! do you want to know how...",[computer science and technology],[S_13],[central south university],[central south university located changsha cit...,[integration knowledge action practice world],"[T_3915, T_3911, T_5426, T_5427, T_5429]","[liu wei, liu yongwei, lin jie, guo pengfei, c...",[liu wei male born hengnan county hunan provin...,"[associate professor, lecturer, lecturer, lect...","[central south university, central south unive...","[V_1622637, V_1622640, V_1622622, V_1622571, V...","[0B63D5E3D4D034E89C33DC5901307461, 0B6DEBC0F80...","[466.575, 402.325, 476.65, 1025.9, 755.075, 42...","[Ex_1622651, Ex_1622617, Ex_1622586, Ex_162263...","[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1...",None,None,None,None,696,1366
3,C_1025076,advanced roller skating tutorial,no prerequisite,the true meaning of experiencing roller skatin...,[computer science and technology],[S_72],[northeastern university],[northeastern university northeastern universi...,[selfimprovement virtue carry thing],[T_5394],[li zhongshan],[fulltime teacher roller skating program north...,[lecturer],[northeastern university],None,None,None,"[Ex_1622870, Ex_1622876, Ex_1622875, Ex_1622889]","[1, 1, 15, 1]",None,None,None,None,2,165
4,C_1025079,scientific fitness method-silk-winding fitness...,no prerequisite,silk wrapping skills are the chinese solution ...,[computer science and technology],[S_97],[beijing sport university],[beijing sport university referred beijing spo...,[selfimprovement virtue carry thing],[T_7939],[wu dong],[professor phd education phd supervisor famous...,[professor],[beijing sport university],"[V_1623000, V_1622990, V_1622999, V_1623003, V...","[0805D199A89E1D799C33DC5901307461, 1026CDC4B72...","[671.97, 925.17, 748.11, 644.25, 543.93, 546.3...","[Ex_1623011, Ex_1622993, Ex_1623006, Ex_162302...","[5, 5, 5, 5, 5]",None,None,None,None,125,688


In [15]:
course_df.head()

,course_id,course_name,course_prerequisites,course_about,course_field,school_id,school_name,school_about,school_motto,teacher_id,teacher_name,teacher_about,teacher_job_title,teacher_org_name,video_id,video_ccid,video_duration,exercise_id,exercise_num_problem,concept_id,concept_prerequisite,concept_name,concept_field,course_total_comments,count_course
0,C_1017355,introduction to traditional chinese medicine e...,the main subjects are college students who hav...,traditional chinese medicine culture - the tre...,[computer science and technology],[S_33],[guangzhou university traditional chinese medi...,[guangzhou university chinese medicine first b...,[selfimprovement virtue carry thing],"[T_7970, T_7971, T_5826, T_7978, T_7979]","[su hong, liu shuaishuai, zhang xiaohu, liu, m...",[master english linguistics currently dean sch...,"[associate professor, lecturer, associate prof...",[guangzhou university traditional chinese medi...,None,None,None,"[Ex_1617861, Ex_1736572, Ex_1617873, Ex_161788...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...",None,None,None,None,39624,1940
1,C_1017419,smart government talk,no prerequisite,this course combines specific cases of traffic...,[computer science and technology],[S_52],[central south university nationality],[southcentral minzu university referred scmzu ...,[selfimprovement virtue carry thing],"[T_7849, T_16199, T_16200, T_16201, T_16202, T...","[hu xinli, wu kaisong, peng qingjun, fang fuji...",[doctor management associate professor directo...,"[associate professor, professor, professor, as...","[central south university nationality, central...",None,None,None,"[Ex_1618263, Ex_1618257, Ex_1618248, Ex_161826...","[15, 15, 20, 15, 15, 15, 15, 16, 15]",None,None,None,None,609,128
2,C_1025064,principles of ideological and political education,no prerequisite,"hello, dear classmate! do you want to know how...",[computer science and technology],[S_13],[central south university],[central south university located changsha cit...,[integration knowledge action practice world],"[T_3915, T_3911, T_5426, T_5427, T_5429]","[liu wei, liu yongwei, lin jie, guo pengfei, c...",[liu wei male born hengnan county hunan provin...,"[associate professor, lecturer, lecturer, lect...","[central south university, central south unive...","[V_1622637, V_1622640, V_1622622, V_1622571, V...","[0B63D5E3D4D034E89C33DC5901307461, 0B6DEBC0F80...","[466.575, 402.325, 476.65, 1025.9, 755.075, 42...","[Ex_1622651, Ex_1622617, Ex_1622586, Ex_162263...","[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1...",None,None,None,None,696,1366
3,C_1025076,advanced roller skating tutorial,no prerequisite,the true meaning of experiencing roller skatin...,[computer science and technology],[S_72],[northeastern university],[northeastern university northeastern universi...,[selfimprovement virtue carry thing],[T_5394],[li zhongshan],[fulltime teacher roller skating program north...,[lecturer],[northeastern university],None,None,None,"[Ex_1622870, Ex_1622876, Ex_1622875, Ex_1622889]","[1, 1, 15, 1]",None,None,None,None,2,165
4,C_1025079,scientific fitness method-silk-winding fitness...,no prerequisite,silk wrapping skills are the chinese solution ...,[computer science and technology],[S_97],[beijing sport university],[beijing sport university referred beijing spo...,[selfimprovement virtue carry thing],[T_7939],[wu dong],[professor phd education phd supervisor famous...,[professor],[beijing sport university],"[V_1623000, V_1622990, V_1622999, V_1623003, V...","[0805D199A89E1D799C33DC5901307461, 1026CDC4B72...","[671.97, 925.17, 748.11, 644.25, 543.93, 546.3...","[Ex_1623011, Ex_1622993, Ex_1623006, Ex_162302...","[5, 5, 5, 5, 5]",None,None,None,None,125,688


In [16]:
def fill_attributes_by_id(course_df, id_col, attr_cols):
    # Merge
    df = course_df[[id_col]+attr_cols].dropna().explode([id_col]+attr_cols)
    df = df.drop_duplicates()
    rel_df = course_df[['course_id',id_col]].explode(id_col)
    df = pd.merge(rel_df,df,on=id_col,how='left')
    df = df.groupby('course_id').agg(list).reset_index()
    course_df = course_df.drop([id_col]+attr_cols,axis=1)
    course_df = pd.merge(course_df,df,on='course_id',how='left')
    return course_df
# Bổ sung thông tin trường học
def fill(course_df,entities):
    for ent in entities:
        cols = course_df.columns[course_df.columns.str.startswith(ent)].tolist()
        attr_cols = [item for item in cols if not (isinstance(item, str) and item.endswith('_id'))]
        id_col = ent+'_id'
        course_df = fill_attributes_by_id(course_df,id_col,attr_cols)
    return course_df
entities=['school','teacher']
course_df = fill(course_df,entities)

In [17]:
print('Sau khi điền bằng top các giá trị xuất hiện nhiều nhất:')
null_stats = compute_null_stats(course_df)
display(null_stats)

Sau khi điền bằng top các giá trị xuất hiện nhiều nhất:


,column,isnull_rate,effectively_null_rate
0,course_id,0.000000,0.0
1,course_name,0.000000,0.0
2,course_prerequisites,0.000000,0.0
3,course_about,0.000000,0.0
4,course_field,0.000000,0.0
5,video_id,0.532538,0.0
6,video_ccid,0.532538,0.0
7,video_duration,0.532538,0.0
8,exercise_id,0.022054,0.0
9,exercise_num_problem,0.022054,0.0


In [18]:
def fill_numcol_null(df, col, count_col='count_course'):
    """
    Điền khuyết cho cột `col` bằng giá trị có tổng `count_col` lớn nhất.
    
    Args:
        df (pd.DataFrame): DataFrame cần xử lý.
        col (str): Tên cột cần điền khuyết.
        count_col (str): Cột đếm dùng để tính tổng (mặc định là 'course_count').

    Returns:
        pd.DataFrame: DataFrame sau khi điền khuyết.
    """
    entity_df=df[[col,count_col]].dropna().explode(col)
    # Tìm giá trị xuất hiện nhiều nhất dựa trên tổng count_col
    weighted_sum = (entity_df[col] * entity_df[count_col]).sum()
    total_count = entity_df[count_col].sum()
    value = weighted_sum / total_count
    print(f"Giá trị điền khuyết cho {col}: {value:.6f}")
    # Điền khuyết
    df[col] = df[col].apply(lambda x: fill_missing_in_list(x, value=round(value,2)))

In [19]:
# fill_numcol_null(course_df,'video_duration')
# print('Sau khi điền:')
# null_stats = compute_null_stats(course_df)
# display(null_stats)

In [20]:
course_info_cols = course_df.drop(['course_id','count_course'],axis=1).columns.tolist()
user_interact = data.drop(course_info_cols,axis=1)

In [21]:
user_interact.head()

,user_id,course_id,enroll_time,user_gender,watch_id,total_watch_time,actual_watch_time,doing_exercise_id,doing_num_correct,user_course_num_comment,user_course_num_replies
0,U_24,C_597314,1560360127,1,NaN,NaN,NaN,NaN,NaN,0,0
1,U_24,C_597208,1560784961,1,NaN,NaN,NaN,NaN,NaN,0,0
2,U_24,C_682129,1570379031,1,NaN,NaN,NaN,NaN,NaN,0,0
3,U_24,C_674910,1570557064,1,NaN,NaN,NaN,NaN,NaN,0,0
4,U_24,C_707038,1570971985,1,NaN,NaN,NaN,NaN,NaN,0,0


In [22]:
user_interact.isnull().mean()

user_id                    0.000000
course_id                  0.000000
enroll_time                0.000000
user_gender                0.000000
watch_id                   0.999437
total_watch_time           0.999437
actual_watch_time          0.999437
doing_exercise_id          0.996887
doing_num_correct          0.996887
user_course_num_comment    0.000000
user_course_num_replies    0.000000
dtype: float64

In [23]:
for col in user_interact.select_dtypes(include='object').columns:
    if user_interact[col].dropna().apply(lambda x: isinstance(x, list)).any():
        user_interact[col] = user_interact[col].apply(lambda x: [] if x else x)

In [24]:
user_course = data[['user_id','course_id']]
del data
data = pd.merge(user_course,course_df,on='course_id',how='left')
data = pd.merge(data,user_interact,on=['user_id','course_id'],how='left')

In [25]:
data.head()

,user_id,course_id,course_name,course_prerequisites,course_about,course_field,video_id,video_ccid,video_duration,exercise_id,exercise_num_problem,concept_id,concept_prerequisite,concept_name,concept_field,course_total_comments,count_course,school_id,school_name,school_about,school_motto,teacher_id,teacher_name,teacher_about,teacher_job_title,teacher_org_name,enroll_time,user_gender,watch_id,total_watch_time,actual_watch_time,doing_exercise_id,doing_num_correct,user_course_num_comment,user_course_num_replies
0,U_24,C_597314,university chinese—analysis of northern song d...,no prerequisite,"at present, the talent gap in data science and...",[chinese language and literature],"[V_10741, V_10729, V_10669, V_10636, V_10583, ...","[059F9E3D0C61EC3A9C33DC5901307461, 05BA5BEC636...","[678.88, 1861.28, 280.63, 1488.369, 1192.15, 6...","[Ex_10607, Ex_10647, Ex_10628, Ex_10711, Ex_10...","[1, 6, 4, 2, 4, 3, 2, 6, 3, 4, 5, 6, 4, 2, 2, ...","[K_D_中国语言文学, K_爱国豪情_中国语言文学, K_爱国将领_中国语言文学, K_爱...","[no prerequisite, no prerequisite, no prerequi...","[d, patriotic pride, patriotic general, patrio...","[chinese language and literature, chinese lang...",0,48,"[S_1, S_14]","[tsinghua university, southeast university]",[referred tsinghua short directly ministry edu...,"[selfimprovement virtue carry thing, stop high...",[T_113],[wang bugao],[wang bugao yangzhong jiangsu studied nanjing ...,[distinguished professor],[tsinghua university],1560360127,1,[],[],[],[],[],0,0
1,U_24,C_597208,data mining: theory and algorithm,no prerequisite,the most interesting theory + the most useful ...,[computer science and technology],"[V_3025, V_2985, V_2964, V_3074, V_3082, V_306...","[03D205A6AEB0B16F9C33DC5901307461, 03F5AF209FA...","[1003.9, 773.66, 886.35, 505.57, 889.61, 930.6...","[Ex_2994, Ex_3099, Ex_3066, Ex_3021, Ex_3006, ...","[5, 3, 5, 3, 5, 6, 3, 3, 5, 5, 4, 5, 5, 2, 5, ...","[K_0 part_计算机科学与技术, K_2 cities_计算机科学与技术, K_2d ...","[no prerequisite, no prerequisite, no prerequi...","[part, cities, 2d space, matrix, + 2, 3d infor...","[computer science and technology, computer sci...",0,6,[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,[selfimprovement virtue carry thing],[T_28],[yuan bo],[doctor computer science university queensland...,[associate researcher],[tsinghua university],1560784961,1,[],[],[],[],[],0,0
2,U_24,C_682129,software theory basics,no prerequisite,"this course introduces formal language, automa...",[computer science and technology],"[V_265396, V_265411, V_265404, V_265442, V_265...","[00784179599D8C659C33DC5901307461, 036F26D5314...","[1236.79, 1212.67, 1584.55, 1574.35, 376.79, 5...","[Ex_265490, Ex_265468, Ex_265398, Ex_265475, E...","[4, 5, 5, 6, 6, 6, 7, 7, 6, 6, 6, 5, 7, 7, 6, 5]","[K_ASCII_计算机科学与技术, K_cfl的判定性质_计算机科学与技术, K_CYK算...","[no prerequisite, no prerequisite, no prerequi...","[ascii, determination of the nature of cfl, cy...","[computer science and technology, computer sci...",13,4016,[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,[selfimprovement virtue carry thing],[T_2648],[luo guiming],[luo guiming received master degree university...,[professor],[tsinghua university],1570379031,1,[],[],[],[],[],0,0
3,U_24,C_674910,pattern aesthetics and creation,no prerequisite,[national premium course] this course provides...,[computer science and technology],None,None,None,"[Ex_35603, Ex_35595, Ex_35599, Ex_35583, Ex_35...","[1, 1, 2, 3, 1, 1, 3, 2, 2, 2, 3]",None,None,None,None,386,7192,[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,[selfimprovement virtue carry thing],[T_307],[nie yuehua],[nie yuehua member chinese artist association ...,[associate professor],[tsinghua university],1570557064,1,[],[],[],[],[],0,0
4,U_24,C_707038,organic chemistry,no prerequisite,the world's organic compounds are rich and col...,"[chemical engineering and technology, chemical]","[V_566855, V_567087, V_566826, V_567004, V_566...","[001EF0F9EA519E1B9C33DC5901307461, 00

In [26]:
data.isnull().mean()

user_id                    0.000000
course_id                  0.000000
course_name                0.000000
course_prerequisites       0.000000
course_about               0.000000
course_field               0.000000
video_id                   0.379174
video_ccid                 0.379174
video_duration             0.379174
exercise_id                0.019379
exercise_num_problem       0.019379
concept_id                 0.589986
concept_prerequisite       0.589986
concept_name               0.589986
concept_field              0.589986
course_total_comments      0.000000
count_course               0.000000
school_id                  0.000000
school_name                0.000000
school_about               0.000000
school_motto               0.000000
teacher_id                 0.000000
teacher_name               0.000000
teacher_about              0.000000
teacher_job_title          0.000000
teacher_org_name           0.000000
enroll_time                0.000000
user_gender                0

In [27]:
data.to_hdf('dataver3.h5',key='df',mode='w')

/tmp/ipykernel_13/1497125970.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index(['user_id', 'course_id', 'course_name', 'course_prerequisites',
       'course_about', 'course_field', 'video_id', 'video_ccid',
       'video_duration', 'exercise_id', 'exercise_num_problem', 'concept_id',
       'concept_prerequisite', 'concept_name', 'concept_field', 'school_id',
       'school_name', 'school_about', 'school_motto', 'teacher_id',
       'teacher_name', 'teacher_about', 'teacher_job_title',
       'teacher_org_name', 'watch_id', 'total_watch_time', 'actual_watch_time',
       'doing_exercise_id', 'doing_num_correct'],
      dtype='object')]

  data.to_hdf('dataver3.h5',key='df',mode='w')
